In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

In [152]:
df = pd.read_csv('new_preprocessed.csv')

In [153]:
df = df.drop(['Unnamed: 0'], axis=1)

In [155]:
y = df['traffic_volume']
X = df.drop(['traffic_volume'], axis=1)

In [145]:
#df = X.join(y, how='inner')

In [88]:
# choose a number of time steps
n_steps =3
# convert into input/output
X, y = split_sequences(df.values, n_steps)
print(X.shape, y.shape)

(40553, 3, 16) (40553,)


In [157]:
# train test splitting
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

In [159]:
# scaling input variables
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [161]:
# scaling output variables
Y_train = y_train.values.reshape(len(y_train), 1)
Y_test = y_test.values.reshape(len(y_test), 1)

Y_train = scaler.fit_transform(Y_train)
Y_test = scaler.transform(Y_test)

In [169]:
# split a multivariate sequence into samples
def split_sequences(x_data, y_data, n_steps):
    # prepare the list for the transformed data
	X, y = list(), list()
    # loop of the entire data
	for i in range(len(x_data)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(x_data):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = x_data[i:end_ix], y_data[end_ix-1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [170]:
# choose a number of time steps
n_steps =3
# convert into input/output for training set
X_train_3D, Y_train_3D = split_sequences(X_train, Y_train, n_steps)
print(X_train_3D.shape, Y_train_3D.shape)

(28386, 3, 16) (28386, 1)


In [171]:
# convert into input/output for test set
X_test_3D, Y_test_3D = split_sequences(X_test, Y_test, n_steps)
print(X_test_3D.shape, Y_test_3D.shape)

(12165, 3, 16) (12165, 1)


In [189]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# initialize the neural network based on LSTM
model = Sequential()
# adding 1st LSTM layer
model.add(LSTM(units=100, return_sequences=True, input_shape=(num_steps, num_features)))
# adding 2nd LSTM layer
model.add(LSTM(units=50, return_sequences=False))
# adding dropout
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=1, activation='linear'))
# compiling the neural network
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=10, verbose=1)

history = model.fit(X_train_3D, Y_train_3D, shuffle=True, epochs=50, callbacks=[es, rlr], validation_split=0.3, verbose=1, batch_size=500)

Epoch 1/50
40/40 [==============================] - 6s 45ms/step - loss: 0.7865 - val_loss: 0.3391
Epoch 2/50
40/40 [==============================] - 1s 22ms/step - loss: 0.2835 - val_loss: 0.1569
Epoch 3/50
40/40 [==============================] - 1s 24ms/step - loss: 0.1547 - val_loss: 0.1090
Epoch 4/50
40/40 [==============================] - 1s 23ms/step - loss: 0.1143 - val_loss: 0.0896
Epoch 5/50
40/40 [==============================] - 1s 21ms/step - loss: 0.0945 - val_loss: 0.0849
Epoch 6/50
40/40 [==============================] - 1s 22ms/step - loss: 0.0881 - val_loss: 0.0852
Epoch 7/50
40/40 [==============================] - 1s 22ms/step - loss: 0.0815 - val_loss: 0.0754
Epoch 8/50
40/40 [==============================] - 1s 21ms/step - loss: 0.0764 - val_loss: 0.0754
Epoch 9/50
40/40 [==============================] - 1s 21ms/step - loss: 0.0736 - val_loss: 0.0752
Epoch 10/50
40/40 [==============================] - 1s 21ms/step - loss: 0.0681 - val_loss: 0.0725
Epoch 11/

In [195]:
# LSTM prediction
LSTM_predTrain = model.predict(X_train_3D)
print('training score =',r2_score(Y_train_3D, LSTM_predTrain))
LSTM_predTest = model.predict(X_test_3D)
print('R2 =',r2_score(LSTM_predTest , Y_test_3D))
print('MAE =',mean_absolute_error(LSTM_predTest , Y_test_3D))
print('RMSE =',mean_squared_error(LSTM_predTest , Y_test_3D, squared=False))

training score = 0.9619683329801219
R2 = 0.9287311658429825
MAE = 0.17714949717759446
RMSE = 0.2638803057735535


In [192]:
LSTM_predTest

array([[1.3132216 ],
       [0.4336381 ],
       [0.38535717],
       ...,
       [1.3347505 ],
       [0.7074195 ],
       [1.2235371 ]], dtype=float32)

In [181]:
Y_test[:-2].shape

(12165, 1)

In [7]:
win_length = 720
batch_size = 32
num_features = x_train.shape[1]
train_generator = TimeseriesGenerator(X_train, Y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(X_test, Y_test, length=win_length, sampling_rate=1, batch_size=batch_size)

In [8]:
# Build LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape= (win_length, num_features), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 720, 128)          74752     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 720, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 720, 128)          131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 720, 128)          0         
_________________________________________________________________
dropout (Dropout)            (None, 720, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit_generator(train_generator, epochs=50, validation_data=test_generator, shuffle=False, callbacks=[early_stopping])

Epoch 1/50
865/865 [==============================] - 1462s 2s/step - loss: 1.0098 - mean_absolute_error: 0.8796 - val_loss: 1.0150 - val_mean_absolute_error: 0.8835
Epoch 2/50
865/865 [==============================] - 1432s 2s/step - loss: 1.0078 - mean_absolute_error: 0.8795 - val_loss: 1.0144 - val_mean_absolute_error: 0.8840
Epoch 3/50
865/865 [==============================] - 1457s 2s/step - loss: 1.0074 - mean_absolute_error: 0.8793 - val_loss: 1.0150 - val_mean_absolute_error: 0.8838
Epoch 4/50
865/865 [==============================] - 1566s 2s/step - loss: 1.0069 - mean_absolute_error: 0.8793 - val_loss: 1.0148 - val_mean_absolute_error: 0.8841
Epoch 5/50
865/865 [==============================] - 1423s 2s/step - loss: 1.0065 - mean_absolute_error: 0.8791 - val_loss: 1.0155 - val_mean_absolute_error: 0.8844
Epoch 6/50
865/865 [==============================] - 1357s 2s/step - loss: 1.0052 - mean_absolute_error: 0.8782 - val_loss: 1.0165 - val_mean_absolute_error: 0.8844
Epoc

In [12]:
model.evaluate_generator(test_generator, verbose=0)

[1.1435537338256836, 0.915569543838501]

In [13]:
predictions = model.predict_generator(test_generator)

In [35]:
X_test[:]

(12167, 17)

In [23]:
X_test[:,0:][win_length:]

array([[ 0.80260895,  0.17137926,  0.74414374, ..., -0.36271527,
        -1.10607761, -0.49288576],
       [ 0.03984344,  0.17137926,  1.0751205 , ..., -0.07458518,
         1.20996275, -0.49288576],
       [-0.36083386,  0.17137926,  0.66235502, ...,  0.06947987,
         0.4379493 , -0.49288576],
       ...,
       [ 0.71006728,  0.17137926,  0.63713046, ..., -0.50678032,
         0.4379493 , -0.49288576],
       [ 0.75413818,  0.17137926,  1.39157403, ...,  0.06947987,
         0.4379493 , -0.49288576],
       [-0.44551345,  0.17137926,  0.76172449, ...,  0.64574006,
         0.4379493 ,  2.23454117]])

In [27]:
data_pred = pd.concat([pd.DataFrame(predictions), pd.DataFrame(X_test[:,0:][win_length:]), pd.DataFrame(Y_test[:,0:][win_length:])], axis=1)

In [45]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#print('R2 =',r2_score(predictions , Y_test[0:11447]))
print('MAE =',mean_absolute_error(predictions , Y_test[0:11447]))
print('RMSE =',mean_squared_error(predictions , Y_test[0:11447], squared=False))

MAE = 0.914038168519139
RMSE = 1.0680035000451993


In [76]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [78]:
# choose a number of time steps
n_steps =3
# convert into input/output
X, y = split_sequences(df.values, n_steps)
print(X.shape, y.shape)

(40553, 3, 16) (40553,)


In [62]:
df.columns

Index(['Unnamed: 0', 'holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all',
       'weather_main', 'weather_description', 'traffic_volume', 'year',
       'month', 'day', 'dayofyear', 'weekofyear', 'dayofweek', 'hour',
       'is_weekend', 'peak_hr'],
      dtype='object')

In [71]:
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume,year,month,day,dayofyear,weekofyear,dayofweek,hour,is_weekend,peak_hr
0,900,288.28,0.0,0.0,40,40,24,5545,2012,10,2,276,40,1,9,5.0,1.0
1,900,289.36,0.0,0.0,75,40,2,4516,2012,10,2,276,40,1,10,5.0,1.0
2,900,289.58,0.0,0.0,90,40,19,4767,2012,10,2,276,40,1,11,5.0,1.0
3,900,290.13,0.0,0.0,90,40,19,5026,2012,10,2,276,40,1,12,5.0,1.0
4,900,291.14,0.0,0.0,75,40,2,4918,2012,10,2,276,40,1,13,5.0,1.0


In [127]:
for i in range(16,10000):
    if 32444 % i == 0:
        print(i)

8111
